In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os

# ---------- 创建保存图像的文件夹 ----------
os.makedirs('res2', exist_ok=True)

In [ ]:
# 设置超参数和设备
batch_size = 2048
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 数据加载
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
    ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
    ])),
    batch_size=batch_size, shuffle=False)

print("训练数据形状:", train_loader.dataset.data.shape)
print("测试数据形状:", test_loader.dataset.data.shape)

In [ ]:
# 定义MLP模型
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 10)
        self.drop = nn.Dropout(0.3)
        self.act = nn.GELU()  # 比ReLU更平滑

    def forward(self, x):
        x = self.act(self.bn1(self.fc1(x)))
        x = self.drop(x)
        x = self.act(self.bn2(self.fc2(x)))
        x = self.drop(x)
        x = self.act(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        return x

# 创建模型和优化器
model = MLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
print(model)

In [ ]:
# ---------- 训练循环 ----------
epochs = 10
train_losses = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.view(x.shape[0], -1).to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = F.cross_entropy(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

In [ ]:
# ---------- 绘制并保存损失下降曲线 ----------
plt.figure(figsize=(8,5))
plt.plot(range(1, epochs+1), train_losses, marker='o')
plt.title('Training Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.tight_layout()
plt.savefig('res/loss_curve.png')
plt.close()

In [ ]:
# ---------- 测试模型并生成混淆矩阵 ----------
model.eval()
correct = 0
test_loss = 0
y_true = []
y_pred = []

with torch.no_grad():
    for batch_idx, (x, y) in enumerate(test_loader):
        x, y = x.view(x.shape[0], -1).to(device), y.to(device)
        output = model(x)
        test_loss += F.cross_entropy(output, y)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()

        y_true.extend(y.cpu().numpy())
        y_pred.extend(pred.cpu().numpy())

    test_loss = test_loss / (batch_idx + 1)
    acc = correct / len(test_loader.dataset)
    print('测试结果 - loss:{:.4f}, acc:{:.4f}'.format(test_loss, acc))

In [ ]:
# ---------- 绘制并保存混淆矩阵 ----------
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(10))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title('MNIST Confusion Matrix')
plt.tight_layout()
plt.savefig('res/confusion_matrix.png')
plt.close()

In [ ]:
# ---------- 统计对角线与非对角线元素 ----------
total_correct = np.trace(cm)         # 对角线上的元素和：预测正确的数量
total_incorrect = cm.sum() - total_correct  # 非对角线：预测错误的数量

# 每类正确与错误数量
correct_per_class = np.diag(cm)                # 各类别正确数
incorrect_per_class = cm.sum(axis=1) - correct_per_class  # 各类别错误数

In [ ]:
# ---------- 绘制总体正确/错误柱状图 ----------
plt.figure(figsize=(6,5))
plt.bar(['Correct', 'Incorrect'], [total_correct, total_incorrect], color=['green', 'red'])
plt.title('Overall Prediction Statistics')
plt.ylabel('Number of Samples')
plt.tight_layout()
plt.savefig('res/overall_correct_incorrect.png')
plt.close()

In [ ]:
# ---------- 绘制每个类别的正确/错误柱状图 ----------
x = np.arange(10)
width = 0.35
plt.figure(figsize=(10,6))
plt.bar(x - width/2, correct_per_class, width, label='Correct', color='green')
plt.bar(x + width/2, incorrect_per_class, width, label='Incorrect', color='red')
plt.xlabel('Digit Class')
plt.ylabel('Number of Samples')
plt.title('Per-Class Correct vs Incorrect Predictions')
plt.xticks(x, range(10))
plt.legend()
plt.tight_layout()
plt.savefig('res/per_class_correct_incorrect.png')
plt.close()